In [1]:
print("hello")

hello


In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'c:\\AI\\AI-Chikitsak\\AI-Chikitsak'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_files(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [7]:
extracted_data = load_pdf_files(data_path="Data/")


In [8]:
# extracted_data
# 

In [21]:
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
    chunks = text_splitter.split_documents(extracted_data)
    return chunks





In [22]:
text_chunks = text_splitter(extracted_data)
print("Length of the text chunks: ", len(text_chunks))


Length of the text chunks:  23916


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

from sentence_transformers import SentenceTransformer

In [12]:
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [13]:
embeddings = download_embeddings()


C:\Users\arman\AppData\Local\Temp\ipykernel_5476\272162384.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [14]:
query_result = embeddings.embed_query("Hello World")

print("lenght ", len(query_result))

lenght  384


In [55]:
# query_result

In [15]:
import os

PINECORE_API_KEY = os.getenv("PINECORN")


In [57]:
# PINECORE_API_KEY

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [17]:
pc = Pinecone(api_key=PINECORE_API_KEY)

In [18]:
index_name = "ai-medi"

In [19]:
# create Vector DB
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECORE_API_KEY)

pc.create_index(
  name=index_name,
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

{
    "name": "ai-medi",
    "metric": "cosine",
    "host": "ai-medi-jsy3z1v.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [24]:
# Store to the vector db
os.environ["PINECONE_API_KEY"] = PINECORE_API_KEY

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [28]:
# load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [38]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [39]:
retriever_doc = retriever.invoke("What is Wegener’s granulomatosis")

In [40]:
retriever_doc

[Document(id='32c680b6-3a1a-471c-a348-13631188ce66', metadata={'creationdate': '2004-12-18T18:11:24-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:41:19-06:00', 'page': 327.0, 'page_label': '328', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 5 (T-Z).pdf', 'total_pages': 728.0}, page_content='Suite 1813 South, Falls Church, V A 22041. (703) 931-5852.\nWegener’s Granulomatosis Support Group, Inc. P.O. Box\n28660, Kansas City, MO 64188-8660. (800) 277-9474.\n<http://www.wgsg.org/wgsg>.\nLata Cherath, PhD\nGALE ENCYCLOPEDIA OF MEDICINE 2 3553\nWegener’s granulomatosis'),
 Document(id='858c61ad-c689-4544-b1fd-9deef28084c2', metadata={'creationdate': '2004-12-18T18:11:24-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:41:19-06:00', 'page': 326.0, 'page_label': '327', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 5 (T-Z).pdf', 'total_pages': 728.0}, page_content='Standardization —The proc

In [43]:
OpenAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ['OpenAI_API_KEY'] = OpenAI_API_KEY

In [61]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=OpenAI_API_KEY, temperature=0.5, max_tokens=900)

In [62]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for a Question Answering task "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use a maximum of five sentences and keep the " 
    "answer concise."
    "If you don't know the answer, say that you don't know."
    "\n\n"
    "{context}"
)


In [63]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [64]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [65]:
response = rag_chain.invoke({"input":"Which medicin i have to take to stop acne"})
print(response["answer"])



I cannot provide medical advice. Please consult a doctor or dermatologist for personalized treatment options for your acne.


In [74]:
response = rag_chain.invoke({"input":"What is meansquare Error"})
print(response)
print(response["answer"])

{'input': 'What is meansquare Error', 'context': [Document(id='0318b666-efb0-4c4d-9634-f29f30dee60e', metadata={'creationdate': '2004-12-18T18:11:24-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:41:19-06:00', 'page': 326.0, 'page_label': '327', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 5 (T-Z).pdf', 'total_pages': 728.0}, page_content='Washington, DC 20002-4242. (202) 336-5700. <ttp://\nwww.apa.org>.\nThe Catholic University of America. Washington, DC 20064.\n(800) 464-3742. <http://www.ericae.net>.\nPaula Anne Ford-Martin\nWegener’s granulomatosis\nDefinition\nWegener’s granulomatosis is a very rare disease that\naffects many different organs and systems of the body. It\nKEY TERMS\nNorms—Normative or mean score for a particular\nage group. \nRepresentative sample—A random sample of peo-\nple that adequately represents the test-taking pop-\nulation in age, gender, race, and socioeconomic\nstanding.\nStandard deviation—A measure 